# Ungraded Lab: Intro to Keras Tuner

El desarrollo de modelos de aprendizaje automático suele ser un proceso iterativo. Se comienza con un diseño inicial y luego se reconfigura hasta conseguir un modelo que pueda ser entrenado de forma eficiente en términos de tiempo y recursos computacionales. Como ya sabrás, estos ajustes que realizas se denominan _hiperparámetros_. Son las variables que gobiernan el proceso de entrenamiento y la topología de un modelo ML. Éstas permanecen constantes a lo largo del proceso de entrenamiento y tienen un impacto directo en el rendimiento de su programa de ML. 

El proceso de encontrar el conjunto óptimo de hiperparámetros se denomina *ajuste de hiperparámetros* o *hiperajuste*, y es una parte esencial de un proceso de aprendizaje automático. Sin él, se puede acabar con un modelo que tiene parámetros innecesarios y que tarda demasiado en entrenarse.

Los hiperparámetros son de dos tipos:
1. *Hiperparámetros del modelo* que influyen en la selección del modelo, como el número y la anchura de las capas ocultas

2. 2. *Hiperparámetros del algoritmo* que influyen en la velocidad y la calidad del algoritmo de aprendizaje, como la tasa de aprendizaje en el caso del Descenso Gradiente Estocástico (SGD) y el número de vecinos más cercanos en el caso de un clasificador KNN (k Nearest Neighbors).

En el caso de modelos más complejos, el número de hiperparámetros puede aumentar drásticamente y ajustarlos manualmente puede ser todo un reto.

En este laboratorio, practicarás el ajuste de hiperparámetros con [Keras Tuner](https://keras-team.github.io/keras-tuner/), un paquete del equipo Keras que automatiza este proceso. Para comparar, primero entrenarás un modelo de referencia con hiperparámetros preseleccionados, y luego volverás a realizar el proceso con hiperparámetros ajustados. Algunos de los ejemplos y discusiones aquí son tomados del [tutorial oficial proporcionado por Tensorflow](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb#scrollTo=sKwLOzKpFGAj) pero hemos expuesto algunas partes clave para mayor claridad.

¡Comencemos!

**Nota: Los cuadernos de este curso son compartidos con acceso de sólo lectura. Para poder guardar tu trabajo, por favor, selecciona Archivo > Guardar una copia en Drive en el menú de Colab y ejecuta el cuaderno desde ahí. Necesitarás una cuenta de Gmail para guardar una copia.**

## Descargar y preparar el conjunto de datos

Primero carguemos el [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) en su espacio de trabajo. Lo utilizarás para entrenar un modelo de aprendizaje automático que clasifique imágenes de ropa.

In [ ]:
# Import keras
from tensorflow import keras

In [ ]:
# Download the dataset and split into train and test sets
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

Para el preprocesamiento, normalizará los valores de los píxeles para que el entrenamiento converja más rápidamente.

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Rendimiento de referencia

Como se ha mencionado, primero tendrá un rendimiento de referencia utilizando parámetros elegidos arbitrariamente para poder comparar los resultados más adelante. En interés de los límites de tiempo y recursos proporcionados por Colab, sólo construirá una red neuronal densa superficial (DNN) como se muestra a continuación. Esto es para demostrar los conceptos sin involucrar enormes conjuntos de datos y largos tiempos de sintonización y entrenamiento. Como verás más adelante, incluso los modelos pequeños pueden tardar en afinarse. Puedes ampliar los conceptos aquí cuando llegues a construir modelos más complejos en tus propios proyectos. 

In [ ]:
# Construir el modelo de referencia utilizando la API secuencial
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(28, 28)))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1')) # You will tune this layer later
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(10, activation='softmax'))

# Print model summary
b_model.summary()

Como se muestra, codificamos todos los hiperparámetros al declarar las capas. Estos incluyen el número de unidades ocultas, la activación y el abandono. Verás cómo puedes ajustar automáticamente algunos de ellos un poco más adelante.

A continuación, configuremos la pérdida, las métricas y el optimizador. La tasa de aprendizaje también es un hiperparámetro que se puede ajustar automáticamente, pero por ahora, vamos a establecerla en `0,001`.

In [ ]:
# Setup the training parameters
b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

Con todos los ajustes establecidos, puedes empezar a entrenar el modelo. Hemos fijado el número de épocas en 10, pero no dudes en aumentarlo si tienes más tiempo para pasar por el cuaderno. 

In [ ]:
# Number of training epochs.
NUM_EPOCHS = 10

# Train the model
b_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

Por último, se quiere ver cómo se comporta este modelo de referencia frente al conjunto de pruebas.

In [ ]:
# Evaluate model on the test set
b_eval_dict = b_model.evaluate(img_test, label_test, return_dict=True)

Vamos a definir una función de ayuda para mostrar los resultados, de modo que sea más fácil compararlos después.

In [ ]:
# Definir la función de ayuda
def print_results(model, model_name, layer_name, eval_dict):
  '''
  Imprime los valores de los subparámetros a ajustar, y los resultados de la evaluación del modelo

  Args:
    model (Modelo) - Modelo Keras a evaluar
    model_name (string) - cadena arbitraria para identificar el modelo
    layer_name (string) - nombre de la capa a ajustar
    eval_dict (dict) - resultados de model.evaluate
  '''
  print(f'\n{model_name}:')

  print(f'número de unidades en la primera capa densa: {model.get_layer(layer_name).units}')
  print(f'tasa de aprendizaje para el optimizador: {model.optimizer.lr.numpy()}')

  for key,value in eval_dict.items():
    print(f'{key}: {value}')

# Print results for baseline model
print_results(b_model, 'BASELINE MODEL', 'dense_1', b_eval_dict)

Eso es todo para obtener los resultados de un único conjunto de hiperparámetros. Como puede ver, este proceso puede ser tedioso si quiere probar diferentes conjuntos de parámetros. Por ejemplo, ¿mejorará su modelo si utiliza `tasa de aprendizaje=0.00001` y `unidades=128`? ¿Y si se combina `0,001` con `256`? El proceso será aún más difícil si decides también afinar el dropout y probar también otras funciones de activación. Keras Tuner resuelve este problema al tener una API para buscar automáticamente el conjunto óptimo. Sólo tendrás que configurarlo una vez y esperar los resultados. Verás cómo se hace esto en las siguientes secciones.

## Keras Tuner

Para realizar el hypertuning con Keras Tuner, necesitarás:

* Definir el modelo
* Seleccionar los hiperparámetros a ajustar
* Definir su espacio de búsqueda
* Definir la estrategia de búsqueda

### Instalar e importar paquetes

Comenzará instalando e importando los paquetes necesarios.

In [ ]:
# Install Keras Tuner
!pip install -q -U keras-tuner

In [ ]:
# Import required packages
import tensorflow as tf
import kerastuner as kt

### Definir el modelo

El modelo que se configura para el hipertuning se denomina *hipermodelo*. Cuando construye este modelo, define el espacio de búsqueda de hiperparámetros además de la arquitectura del modelo. 

Puede definir un hipermodelo a través de dos enfoques:

* Utilizando una función de construcción de modelos
* Mediante la [subclase de la clase `HyperModel`](https://keras-team.github.io/keras-tuner/#you-can-use-a-hypermodel-subclass-instead-of-a-model-building-function) de la API Keras Tuner


En este laboratorio, adoptará el primer enfoque: utilizará una función de construcción de modelos para definir el modelo de clasificación de imágenes. Esta función devuelve un modelo compilado y utiliza los hiperparámetros que usted define en línea para hipoajustar el modelo. 

La función que se muestra a continuación básicamente construye el mismo modelo que utilizó anteriormente. La diferencia es que hay dos hiperparámetros que se configuran para el ajuste:

* el número de unidades ocultas de la primera capa densa
* la tasa de aprendizaje del optimizador Adam

Verás que esto se hace con un objeto HyperParameters que configura el hiperparámetro que quieres afinar. Para este ejercicio, lo harás: 

* utilizar su método `Int()` para definir el espacio de búsqueda de las unidades Densas. Esto le permite establecer un valor mínimo y máximo, así como el tamaño del paso cuando se incrementa entre estos valores. 

* Utiliza su método `Choice()` para la tasa de aprendizaje. Esto le permite definir valores discretos para incluir en el espacio de búsqueda cuando se hipertunea.

Puede ver todos los métodos disponibles y su uso de ejemplo en la [documentación oficial](https://keras-team.github.io/keras-tuner/documentation/hyperparameters/#hyperparameters).

In [ ]:
def model_builder(hp):
  '''
  Construye el modelo y establece los hiperparámetros a afinar.

  Args:
    hp - Objeto Keras tuner

  Devuelve:
    El modelo con los hiperparámetros a afinar
  '''
  
  # Inicializar la API secuencial y empezar a apilar las capas
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Ajuste el número de unidades en la primera capa densa
  # Elija un valor óptimo entre 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu', name='tuned_dense_1'))

  # Añadir las siguientes capas
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10, activation='softmax'))

  # Ajuste la tasa de aprendizaje para el optimizador
  # Elija un valor óptimo entre 0,01, 0,001 o 0,0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

## Instanciar el sintonizador y realizar el hipertuning

Ahora que tienes el constructor del modelo, puedes definir cómo el sintonizador puede encontrar el conjunto óptimo de hiperparámetros, también llamado estrategia de búsqueda. Keras Tuner has [four tuners](https://keras-team.github.io/keras-tuner/documentation/tuners/) disponible con estrategias incorporadas - `RandomSearch`, `Hyperband`, `BayesianOptimization`, y `Sklearn`. 

En este tutorial, se utilizará el sintonizador Hyperband. El Hyperband es un algoritmo desarrollado específicamente para la optimización de hiperparámetros. Utiliza la asignación adaptativa de recursos y la parada temprana para converger rápidamente en un modelo de alto rendimiento. Para ello, se utiliza un campeonato deportivo en el que el algoritmo entrena a un gran número de modelos durante unas pocas épocas y lleva a la siguiente ronda sólo a la mitad de los modelos con mejores resultados. 

Puede leer sobre la intuición detrás del algoritmo en la sección 3 de [this paper](https://arxiv.org/pdf/1603.06560.pdf).

Hyperband determina el número de modelos a entrenar en un paréntesis calculando 1 + log<sub>`factor`</sub>(`max_epochs`) y redondeándolo al entero más cercano. You will see these parameters (i.e. `factor` and `max_epochs` passed into the initializer below). Además, también necesitarás definir lo siguiente para instanciar el sintonizador Hyperband:

* el hipermodelo (construido por su función de construcción de modelos)
* el "objetivo" a optimizar (por ejemplo, la precisión de la validación)
* un "directorio" para guardar los registros y puntos de control de cada prueba (configuración del modelo) ejecutada durante la búsqueda de hiperparámetros. Si se vuelve a ejecutar la búsqueda de hiperparámetros, el Afinador Keras utiliza el estado existente de estos registros para reanudar la búsqueda. Para desactivar este comportamiento, pase un argumento adicional `overwrite=True` al instanciar el sintonizador.
* el `nombre_del_proyecto` para diferenciarlo de otras ejecuciones. Se utilizará como nombre de subdirectorio bajo el `directorio`.

Puedes consultar la [documentación](https://keras.io/api/keras_tuner/tuners/hyperband/) para conocer otros argumentos que puedes pasar.

In [ ]:
# Instantiate the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='kt_dir',
                     project_name='kt_hyperband')

Veamos un resumen de los hiperparámetros que se van a sintonizar:

In [ ]:
# Display hypertuning settings
tuner.search_space_summary()

Puedes pasar una llamada de retorno para detener el entrenamiento antes de tiempo cuando una métrica no está mejorando. A continuación, definimos una devolución de llamada [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) para supervisar la pérdida de validación y detener el entrenamiento si no mejora después de 5 épocas.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Ahora se ejecutará la búsqueda de hiperparámetros. Los argumentos para el método de búsqueda son los mismos que los utilizados para `tf.keras.model.fit` además del callback anterior. Esto tomará alrededor de 10 minutos para ejecutar.

In [ ]:
# Perform hypertuning
tuner.search(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early])

Puede obtener el modelo de mayor rendimiento con el método [get_best_hyperparameters()](https://keras-team.github.io/keras-tuner/documentation/tuners/#get_best_hyperparameters-method).

In [ ]:
# Obtener los hiperparámetros óptimos a partir de los resultados
best_hps=tuner.get_best_hyperparameters()[0]

print(f"""
La búsqueda de hiperparámetros está completa. El número óptimo de unidades en la primera capa densamente conectada
es {best_hps.get('units')} y la tasa de aprendizaje óptima para el optimizador
es {best_hps.get('learning_rate')}.
""")

## Construir y entrenar el modelo

Ahora que tiene el mejor conjunto de hiperparámetros, puede reconstruir el hipermodelo con estos valores y volver a entrenarlo.

In [ ]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

In [ ]:
# Train the hypertuned model
h_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

A continuación, obtendrá su rendimiento frente al conjunto de pruebas.

In [ ]:
# Evaluate the hypertuned model against the test set
h_eval_dict = h_model.evaluate(img_test, label_test, return_dict=True)

Podemos comparar los resultados obtenidos con el modelo de referencia que utilizamos al principio del cuaderno. Los resultados pueden variar, pero normalmente se obtiene un modelo que tiene menos unidades en la capa densa, mientras que tiene una pérdida y precisión comparables. Esto indica que redujiste el tamaño del modelo y ahorraste recursos de computación mientras seguías teniendo más o menos la misma precisión.

In [ ]:
# Impresión de los resultados del modelo de referencia y del modelo hipertecnificado
print_results(b_model, 'BASELINE MODEL', 'dense_1', b_eval_dict)
print_results(h_model, 'HYPERTUNED MODEL', 'tuned_dense_1', h_eval_dict)

## Desafíos extra (opcional)

Si quieres seguir practicando con Keras Tuner en este cuaderno, puedes hacer un reset de fábrica (`Runtime > Factory reset runtime`) y enfrentarte a cualquiera de los siguientes:

- hipertunear la capa de abandono con `hp.Float()` o `hp.Choice()`
- Hiperajustar la función de activación de la 1ª capa densa con `hp.Choice()`.
- determinar el número óptimo de capas densas que puede añadir para mejorar el modelo. Puedes usar el código [aquí](https://keras.io/guides/keras_tuner/getting_started/#the-search-space-may-contain-conditional-hyperparameters) como referencia.
- explorar las clases predefinidas `HyperModel` - [HyperXception e HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) para aplicaciones de visión por ordenador.

## Resumen

En este tutorial, usted utilizó el Afinador Keras para afinar convenientemente los hiperparámetros. Se ha definido cuáles son los que hay que ajustar, el espacio de búsqueda y la estrategia de búsqueda para llegar al conjunto óptimo de hiperparámetros. Estos conceptos se discutirán de nuevo en las próximas secciones, pero en el contexto de AutoML, un paquete que automatiza todo el proceso de aprendizaje automático. A continuación.
